# Testing

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import joblib

from getScreenTime import get_screen_time_data
from getAppName import get_app_name

45


## get data

In [10]:
#gets (appname, (year, month, hour, minute, second, dayOfWeek))
data = get_screen_time_data(758000000, 758935000) # jan 7 10pm - January 18, January 18, 2025 6:16:40 PM

# appname to index
apps = get_app_name(758000000, 758935000)

len_data = len(data)

X_test = np.zeros((len_data, 4))
Y_test = np.zeros((len_data, 100))

for i in range(len_data):
    x = data[i][1]
    y = data[i][0]

    X_test[i][0] = x[1]
    X_test[i][1] = x[2]
    X_test[i][2] = x[3]
    X_test[i][3] = x[5]

    if y not in apps:
        Y_test[i][99] = 1
    else:
        ind = apps.index(y)
        Y_test[i][ind] = 1





print(X_test)
print(Y_test)





[[ 1. 22. 33.  2.]
 [ 1. 23.  9.  2.]
 [ 1. 23. 38.  2.]
 ...
 [ 1. 17.  3.  6.]
 [ 1. 17. 29.  6.]
 [ 1. 17. 59.  6.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dense(100, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_test, Y_test, epochs=10)